# 01 - Data Consolidation

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import pandas as pd

import stock_utils.constants as cnst
import stock_utils.data as sd

pd.set_option('display.min_rows', 32)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
SCREENER_EPS_DATA_DIR = PosixPath('../data/Screener.in/EPS') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Consolidating historical stock data

### All available symbols

In [3]:
stock_symbols = sd.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['DEEPAKFERT',
 'HDFCBANK',
 'HDFCSML250',
 'INDIGOPNTS',
 'ITBEES',
 'JUBLFOOD',
 'NH']

### Consolidated data sample

In [4]:
sd.get_consolidated_stock_data(
    stock_symbols[1],
    cnst.NSE_DATA_DIR,
    cnst.SCREENER_EPS_DATA_DIR
)

,Date,series,OPEN,HIGH,LOW,PREV. CLOSE,ltp,close,vwap,52W H,52W L,VOLUME,VALUE,No of trades,PE
0,2020-01-01,EQ,1276.10,1280.00,1270.60,1272.10,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,31.185
1,2020-01-02,EQ,1279.00,1288.00,1279.00,1278.60,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,31.384
2,2020-01-03,EQ,1282.20,1285.00,1263.60,1286.75,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,30.937
3,2020-01-06,EQ,1260.00,1261.80,1236.00,1268.40,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,30.267
4,2020-01-07,EQ,1258.90,1271.45,1252.25,1240.95,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,30.746
5,2020-01-08,EQ,1246.95,1262.15,1240.05,1260.60,1256.00,1257.30,1253.71,2503.3,1084.00,5666055,7.103612e+09,141795,30.666
6,2020-01-09,EQ,1265.00,1275.80,1263.10,1257.30,1271.80,1271.40,1271.27,2503.3,1084.00,4773158,6.067965e+09,125829,31.010
7,2020-01-10,EQ,1284.10,1286.90,1275.10,1271.40,1281.00,1282.70,1282.11,2503.3,1084.00,4607290,5.907037e+09,123583,31.285
8,2020-01-13,EQ,1282.70,1296.50,1276.00,1282.70,1287.00,1286.00,1289.33,2503.3,1084.00,3725784,4.803756e+09,107121,31.366
9,2020-01-14,EQ,1289.00,1292.55,1277.50,1286.00,1288.00,1289.50,1284.81,2503.3,1084.00,3943145,5.066204e+09,91722,31.451


### Saving consolidated data for all symbols

In [5]:
sd.consolidate_all_stock_data(cnst.NSE_DATA_DIR, cnst.SCREENER_EPS_DATA_DIR)

DEEPAKFERT - 5 files
	965 records
	2020-01-01 to 2024-02-16
	Saved to '../data/NSE/DEEPAKFERT/consolidated.parquet'
HDFCBANK - 5 files
	1122 records
	2020-01-01 to 2024-07-05
	Saved to '../data/NSE/HDFCBANK/consolidated.parquet'
HDFCSML250 - 2 files
	334 records
	2023-02-21 to 2024-06-28
	Saved to '../data/NSE/HDFCSML250/consolidated.parquet'
INDIGOPNTS - 4 files
	775 records
	2021-02-02 to 2024-03-15
	Saved to '../data/NSE/INDIGOPNTS/consolidated.parquet'
ITBEES - 5 files
	915 records
	2020-07-01 to 2024-03-01
	Saved to '../data/NSE/ITBEES/consolidated.parquet'
JUBLFOOD - 5 files
	1018 records
	2020-01-01 to 2024-02-02
	Saved to '../data/NSE/JUBLFOOD/consolidated.parquet'
NH - 5 files
	1074 records
	2020-01-01 to 2024-04-26
	Saved to '../data/NSE/NH/consolidated.parquet'
